## LaMarsh Example 6.1
calculate $f$ and $k_{\infty}$ for a mixture of $^{235}\text{U}$ and sodium in which the uranium is present to 1 w/o

In [1]:
import openmc
import numpy as np

In [2]:
# define material

w_frac_fuel = 0.01;
fuel_nuclide = 'U235';
fuel_rho = 19.1;
fuel_A = 235

#fuel_nuclide = 'U233';
#fuel_rho = 19.1;
#fuel_A = 233


w_frac_Na = 1. - w_frac_fuel;
Na_A = 23.
Na_rho = 0.97;

rho_mix = ((w_frac_Na/Na_rho)+(w_frac_fuel/fuel_rho))**(-1);

#Na_atom_ratio = (w_frac_Na/w_frac_fuel)*(fuel_A/Na_A);


fuel = openmc.Material(name='fuel');
fuel.add_nuclide(fuel_nuclide,w_frac_fuel,percent_type='wo');
fuel.add_element('Na',w_frac_Na,percent_type='wo');
fuel.set_density('g/cm3',rho_mix); # estimate, make this better


materials = openmc.Materials([fuel]);
materials.export_to_xml();


In [3]:
# simple geometry

core_region = openmc.model.RectangularParallelepiped(-1.,1.,
                                                  -1.,1.,
                                                  -1.,1.,
                                                  boundary_type='reflective');
fuel_cell = openmc.Cell();
fuel_cell.region = -core_region;
fuel_cell.fill = fuel;

root_univ = openmc.Universe();
root_univ.add_cells([fuel_cell]);

geometry = openmc.Geometry();
geometry.root_universe = root_univ;

geometry.export_to_xml();

In [4]:
# Tallies

fuel_abs = openmc.Tally(name='fuel_abs');
fuel_abs.scores = ['absorption'];
fuel_abs.nuclides = [fuel_nuclide];

total_abs = openmc.Tally(name='total_abs');
total_abs.scores = ['absorption'];

tallies = openmc.Tallies([fuel_abs,total_abs]);
tallies.export_to_xml();

In [5]:
# establish settings for criticality calculation
settings = openmc.Settings();
settings.run_mode = 'eigenvalue';
settings.particles = 5000;
settings.batches = 100;
settings.inactive = 10;

#box = openmc.stats.Box(lower_left = (-1.,-1.,-1.),
#                      upper_right = (1.,1.,1.),
#                      only_fissionable=True);
#src = openmc.Source(space=box);
point = openmc.stats.Point((0,0,0));
src = openmc.Source(space=point);

settings.source = src;
settings.export_to_xml();

In [6]:
openmc.run()

                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%%
                 #######################     %%%%%%%%%%%%%%%%%
                 #####################

In [7]:
sp = openmc.StatePoint('statepoint.100.h5');
fuel_abs_t = sp.get_tally(name='fuel_abs');
fuel_abs_df = fuel_abs_t.get_pandas_dataframe();
fuel_abs_val = fuel_abs_df['mean'].to_numpy();
print(fuel_abs_val);

total_abs_t = sp.get_tally(name='total_abs');
total_abs_df = total_abs_t.get_pandas_dataframe();
total_abs_val = total_abs_df['mean'].to_numpy();
print(total_abs_val);

[0.68270423]
[1.00265395]


In [8]:
f = fuel_abs_val[0]/total_abs_val[0];

print('f = %5.4f' % f);
dir(sp)

k_eff = sp.k_combined.nominal_value
print('k_eff = %5.4f' % k_eff);

eta = k_eff/f;
print('eta = %5.4f' % eta);

f = 0.6809
k_eff = 1.1019
eta = 1.6183
